In [11]:
import os
import pandas as pd
from datasets import Dataset, DatasetDict, concatenate_datasets


def convert_excel_to_dataset(file_path, val_size=0.05, test_size=0.05):
    """
    엑셀 파일을 pandas로 읽어 train, validation, test Dataset으로 나누는 함수
    """
    df = pd.read_excel(file_path, engine="openpyxl")

    # 엑셀 데이터를 translation 구조로 변환
    translations = []
    for index, row in df.iterrows():
        translations.append(
            {
                "translation": {
                    "ko": row[
                        "SOURCE_SENTENCE"
                    ],  # 'SOURCE_SENTENCE' 컬럼이 한국어 문장
                    "en": row["MT_SENTENCE"],  # 'MT_SENTENCE' 컬럼이 영어 문장
                }
            }
        )

    # 전체 데이터를 train, validation, test로 분리
    full_dataset = Dataset.from_list(translations)

    # 5%는 test로 나누기
    train_valid_test_split = full_dataset.train_test_split(test_size=test_size)
    train_valid_dataset = train_valid_test_split["train"]
    test_dataset = train_valid_test_split["test"]

    # 남은 train_valid에서 5%를 validation으로 나누기
    train_valid_split = train_valid_dataset.train_test_split(test_size=val_size)
    train_dataset = train_valid_split["train"]
    valid_dataset = train_valid_split["test"]

    return train_dataset, valid_dataset, test_dataset


def process_folder_with_file_splits(folder_path, val_size=0.05, test_size=0.05):
    """
    폴더를 재귀적으로 탐색하면서 파일별로 train, validation, test로 나누는 함수
    """
    train_datasets = []
    valid_datasets = []
    test_datasets = []

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".xlsx") or file.endswith(".xls"):
                file_path = os.path.join(root, file)
                print(f"Processing file: {file_path}")

                # 파일에서 바로 train, validation, test로 나누기
                train_dataset, valid_dataset, test_dataset = convert_excel_to_dataset(
                    file_path, val_size=val_size, test_size=test_size
                )

                # 각 세트별로 리스트에 추가
                train_datasets.append(train_dataset)
                valid_datasets.append(valid_dataset)
                test_datasets.append(test_dataset)

    # 각 세트들을 하나의 Dataset으로 결합
    if train_datasets and valid_datasets and test_datasets:
        concatenated_train_dataset = concatenate_datasets(train_datasets)
        concatenated_valid_dataset = concatenate_datasets(valid_datasets)
        concatenated_test_dataset = concatenate_datasets(test_datasets)

        # DatasetDict로 반환
        return DatasetDict(
            {
                "train": concatenated_train_dataset,
                "validation": concatenated_valid_dataset,
                "test": concatenated_test_dataset,
            }
        )
    else:
        raise ValueError("No Excel files found in the folder.")


# 실행 예시 (폴더 경로를 입력하여 실행)
folder_path = "/Users/obov/Downloads/aihub_sample"  # 실제 폴더 경로로 변경
dataset_dict = process_folder_with_file_splits(folder_path)

# 변환된 데이터셋 정보 출력
print(dataset_dict)

Processing file: /Users/obov/Downloads/aihub_sample/신규_한영_예술체육학_sample.xlsx
Processing file: /Users/obov/Downloads/aihub_sample/신규_한영_의약학_sample.xlsx
Processing file: /Users/obov/Downloads/aihub_sample/신규_한영_자연과학_sample.xlsx
Processing file: /Users/obov/Downloads/aihub_sample/신규_한영_복합학_sample.xlsx
Processing file: /Users/obov/Downloads/aihub_sample/신규_한영_사회과학_sample.xlsx
Processing file: /Users/obov/Downloads/aihub_sample/신규_한영_공학_sample.xlsx
Processing file: /Users/obov/Downloads/aihub_sample/신규_한영_인문학_sample.xlsx
Processing file: /Users/obov/Downloads/aihub_sample/신규_한영_농수해양학_sample.xlsx
DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 19098
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 1009
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 1062
    })
})


In [12]:
from datasets import DatasetDict, load_dataset, Dataset
from huggingface_hub import HfApi, HfFolder, Repository, notebook_login

# Hugging Face 로그인 (토큰을 설정하거나 자동으로 로그인)
notebook_login()

# 데이터셋 업로드할 준비된 데이터셋 예시
# dataset_dict = DatasetDict({...}) <- 이전 단계에서 만든 데이터셋을 사용
dataset_dict = dataset_dict  # 위에서 생성한 DatasetDict

# 데이터셋을 업로드할 Hugging Face 데이터셋 리포지토리 이름을 설정
dataset_repo_id = "obov/hf-aihub-sample-translate-en-ko"

# 데이터셋 저장소를 Hugging Face에 업로드하기
dataset_dict.push_to_hub(dataset_repo_id)

print(f"Dataset uploaded to https://huggingface.co/datasets/{dataset_repo_id}")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/20 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Dataset uploaded to https://huggingface.co/datasets/obov/hf-aihub-sample-translate-en-ko
